In [7]:
import requests

def fetch_news_article(api_key):
    url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}'
    response = requests.get(url)
    data = response.json()
    if data['status'] == 'ok' and data['articles']:
        articles = data['articles']
        for article in articles:
            if article['title'] and article['description']:
                return article['title'] + " " + article['description']
    return None

api_key = '249f32a912ef45098f9afeebb871c876'
article = fetch_news_article(api_key)
if article:
    print("Fetched Article:", article)
else:
    print("Failed to fetch an article. Please check your API key and network connection.")


Fetched Article: Americans freed in Russia prisoner swap land in US - BBC.com US President Joe Biden welcomed them off the plane at Joint Base Andrews in Maryland.


In [8]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def nltk_ner(text):
    words = word_tokenize(text)
    tags = pos_tag(words)
    tree = ne_chunk(tags)
    entities = []
    for subtree in tree:
        if isinstance(subtree, nltk.Tree):
            entity = " ".join([word for word, tag in subtree.leaves()])
            entity_type = subtree.label()
            entities.append((entity, entity_type))
    return entities

if article:
    nltk_entities = nltk_ner(article)
    print("NLTK Entities:", nltk_entities)
else:
    print("No article to process.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ebine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\ebine\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ebine\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


NLTK Entities: [('Americans', 'GPE'), ('Russia', 'GPE'), ('US', 'GPE'), ('Joe Biden', 'PERSON'), ('Joint Base Andrews', 'ORGANIZATION'), ('Maryland', 'GPE')]


In [9]:
import spacy

nlp = spacy.load('en_core_web_sm')

def spacy_ner(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

if article:
    spacy_entities = spacy_ner(article)
    print("SpaCy Entities:", spacy_entities)
else:
    print("No article to process.")


SpaCy Entities: [('Americans', 'NORP'), ('Russia', 'GPE'), ('US', 'GPE'), ('US', 'GPE'), ('Joe Biden', 'PERSON'), ('Joint Base Andrews', 'FAC'), ('Maryland', 'GPE')]


In [10]:
def compare_entities(nltk_entities, spacy_entities):
    nltk_set = set(nltk_entities)
    spacy_set = set(spacy_entities)
    
    common = nltk_set & spacy_set
    only_nltk = nltk_set - spacy_set
    only_spacy = spacy_set - nltk_set

    return common, only_nltk, only_spacy

if article:
    common_entities, only_nltk_entities, only_spacy_entities = compare_entities(nltk_entities, spacy_entities)

    print("Common Entities:", common_entities)
    print("Entities only in NLTK:", only_nltk_entities)
    print("Entities only in SpaCy:", only_spacy_entities)
else:
    print("No entities to compare.")


Common Entities: {('Maryland', 'GPE'), ('Joe Biden', 'PERSON'), ('US', 'GPE'), ('Russia', 'GPE')}
Entities only in NLTK: {('Joint Base Andrews', 'ORGANIZATION'), ('Americans', 'GPE')}
Entities only in SpaCy: {('Joint Base Andrews', 'FAC'), ('Americans', 'NORP')}
